In [ ]:

# import

from neo4j import GraphDatabase


In [ ]:

# 连接数据库

URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "#Wwwww3135728")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    print("Connection established.")
    

In [ ]:

# 创建三元组关系
# 图数据库的增、删、改、查都是查询

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    summary = driver.execute_query("""
        CREATE (me:Person {name: $person})
        CREATE (t1:Trait {name: $trait})
        CREATE (me)-[:具有人格类型]->(t1)
        """,
        person="我", trait="系统型构建者",
        database_="neo4j",
    ).summary
    print("Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after
    ))


In [ ]:

# 查询三元组关系
# 给节点赋值变量p，是把查询的节点赋值给变量，后面返回ide时候可以引用。没有赋值变量，将不能引用
# 节点结构（变量：标签 {属性}) ,标签可以一样，查询的时候可以按标签查询。标签也叫属性类型
# 小括号节点、中括号关系、大括号属性

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    records, summary, keys = driver.execute_query("""
        MATCH (p:Person)-[:具有人格类型]->(:Trait)
        RETURN p.name AS name
        """,
        database_="neo4j",
    )
    
    # Loop through results and do something with them
    for record in records:
        print(record.data())  # obtain record as dict
    
    # Summary information
    print("The query `{query}` returned {records_count} records in {time} ms.".format(
        query=summary.query, records_count=len(records),
        time=summary.result_available_after
    ))


In [ ]:

# 更新数据1

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    records, summary, keys = driver.execute_query("""
        MATCH (p:Person {name: $name})
        SET p.age = $age
        """, name="我", age=32,
        database_="neo4j",
    )
    print(f"Query counters: {summary.counters}.")


In [ ]:

# 更新数据2

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    records, summary, keys = driver.execute_query("""
        MATCH (p:Person {name: $name})
        CREATE (t2:Trait {name: $friend})
        CREATE (p)-[:思维风格]->(t2)
        """, name="我", friend="理性而高度结构化",
        database_="neo4j",
    )
    print(f"Query counters: {summary.counters}.")


In [ ]:

# 删除数据

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    # 这不仅会删除p，还会删除它的所有关系
    records, summary, keys = driver.execute_query("""
        MATCH (p:Person {name: $name})
        DETACH DELETE p
        """, name="我",
        database_="neo4j",
    )
    print(f"Query counters: {summary.counters}.")


In [ ]:

# 传入属性1

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(
        "MERGE (:Person {name: $name})",
        name="我", age=32,
        database_="neo4j",
    )


In [ ]:

# 传入属性2（字典）

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    parameters = {
        "name": "我",
        "age": 32
    }
    driver.execute_query(
        "MERGE (:Person {name: $name, age: $age})",
        parameters_=parameters,
        database_="neo4j",
    )


In [ ]:

# 更新关系

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    records, summary, keys = driver.execute_query("""
        MATCH (p:Person {name: $name})
        MATCH (t1:Trait {name: $trait})
        CREATE (p)-[:具有人格类型]->(t1)
        """, name="我", trait="系统型构建者",
        database_="neo4j",
    )
    print(f"Query counters: {summary.counters}.")


In [ ]:

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    records, summary, keys = driver.execute_query("""
        MATCH (p:Person {name: $name})
        MATCH (t2:Trait {name: $trait})
        CREATE (p)-[:思维风格]->(t2)
        """, name="我", trait="理性而高度结构化",
        database_="neo4j",
    )
    print(f"Query counters: {summary.counters}.")
    

In [ ]:

# 获取错误

from neo4j.exceptions import Neo4jError

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    try:
        driver.execute_query('MATCH (p:Person) RETURN', database_='<database-name>')
    except Neo4jError as e:
        print('Neo4j error code:', e.code)
        print('Exception message:', e.message)
    

In [ ]:

# 通过关键字参数来定义execute_query函数的行为，如dtabase_，指定查询的数据库

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(
        "MATCH (p:Person) RETURN p.name",
        database_="neo4j",
    )
    

In [ ]:

# 上面几个函数在创建节点关系的时候就会保存一个只读副本，可以对这个副本进行读取，只读副本请求

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(
        "MATCH (p:Person) RETURN p.name",
        routing_="r",  # short for neo4j.RoutingControl.READ
        database_="neo4j",
    )


In [ ]:

# 操作查询结果1（返回列表）

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    records, summary, keys = driver.execute_query(
        "MATCH (a:Person) RETURN a.name AS name, a.age AS age",
        database_="neo4j",
    )
    for person in records:
        print(person)
        # person["name"] or person["age"] are also valid
    
    # Some summary information
    print("Query `{query}` returned {records_count} records in {time} ms.".format(
        query=summary.query, records_count=len(records),
        time=summary.result_available_after
    ))
    
    print(f"Available keys are {keys}")  # ['name', 'age']


In [ ]:

# 操作查询结果2（返回df数据格式，通过关键字参数）

import neo4j

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    pandas_df = driver.execute_query(
        "UNWIND range(1, 10) AS n RETURN n, n+1 AS m",
        database_="neo4j",
        result_transformer_=neo4j.Result.to_df
    )
    print(type(pandas_df))  # <class 'pandas.core.frame.DataFrame'>


In [6]:

# 操作返回结果（返回图）

import pyvis
from neo4j import GraphDatabase
import neo4j

URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "#Wwwww3135728")

def visualize_result(query_graph, nodes_text_properties):
    visual_graph = pyvis.network.Network()

    for node in query_graph.nodes:
        node_label = list(node.labels)[0]
        node_text = node[nodes_text_properties[node_label]]
        visual_graph.add_node(node.element_id, node_text, group=node_label)

    for relationship in query_graph.relationships:
        visual_graph.add_edge(
            relationship.start_node.element_id,
            relationship.end_node.element_id,
            # label=relationship.type,
            title=relationship.type
        )

    visual_graph.show('network.html', notebook=False)

# Query to get a graphy result

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    graph_result = driver.execute_query("""
        MATCH (n)-[r]->(m) RETURN n, r, m
        """, 
        result_transformer_=neo4j.Result.graph,
    )
    
    # 告诉程序每种节点类型要显示哪个属性字段作为节点文本
    nodes_text_properties = {  # what property to use as text for each node 
        "Person": "name",
        "Trait": "name"
    }
    
    visualize_result(graph_result, nodes_text_properties)


network.html


In [7]:

from IPython.display import IFrame

IFrame('network.html', width=800, height=600)
